In [1]:
pip install git+https://github.com/egoolish/ecp_python

  Cloning https://github.com/egoolish/ecp_python to /private/var/folders/1r/_mt76hv974ddmv82vjxl18pm0000gn/T/pip-req-build-9qvoe5t7
  Stored in directory: /private/var/folders/1r/_mt76hv974ddmv82vjxl18pm0000gn/T/pip-ephem-wheel-cache-687jo4l_/wheels/16/c4/81/5f49ffa704964ad0a2def2f6626cc7e67a718ffa2355d93782
Successfully built ecp
Note: you may need to restart the kernel to use updated packages.


In [49]:
#import needed packages

import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize
import time
from ecp import e_divisive, e_agglomerative
%matplotlib inline
from sklearn.cluster import AgglomerativeClustering as AG

In [50]:
# P is the (k x k) transition matrix
# S is the vector of averages for each state (of length k)
# sigma is the shared variance
# n is the number of points to generate (length of time series)
# if debug = True, return state_gen rather than time_series.
def mkv_to_ts(P, S, sigma = 1, n = 10000, debug=False):
    k = len(S)
    base_matrix = np.tile(S, (n, 1)).T
    noise_matrix = np.random.normal(0, sigma, (k, n))
    full_matrix = base_matrix + noise_matrix
    
    state_gen = np.empty(n, dtype = np.int32)
    state_gen[0] = 0
    for i in range(1, n):
        state_gen[i] = np.random.choice(k, p=P[state_gen[i-1]])
    return state_gen if debug else np.choose(state_gen, full_matrix)

In [51]:
def MCTS(ts, num_of_states, alpha=1):
    labels = AG(num_of_states).fit_predict(np.reshape(ts, (len(ts), 1)))
    ests = [0]
    ests_labels = []
    for i in range(1, len(labels)):
        if labels[i] != labels[i-1]:
            ests.append(i)
            ests_labels.append(labels[i])
    ests.append(len(labels))
    ests_labels.append(labels[-1])
    
    ests = np.array(ests)
    ests_labels = np.array(ests_labels)

    lens = np.diff(ests)
    find_avg_time = np.vectorize(lambda x : np.mean(lens[ests_labels == x]))
    avg_times = find_avg_time(np.arange(num_of_states))

    P_diag = np.vectorize(lambda x : (x - 1)/x)(avg_times)

    est_P = np.zeros((num_of_states, num_of_states)) + alpha
    for i in range(1, len(labels)):
        est_P[labels[i-1], labels[i]] += 1
    np.fill_diagonal(est_P, 0)
    est_P = normalize(est_P, axis = 1, norm='l1')
    est_P = (est_P.T*(1 - P_diag)).T
    np.fill_diagonal(est_P, P_diag)
    
    est_S = np.vectorize(lambda x: np.mean(ts[labels == x]))(np.arange(num_of_states))

    return est_P, est_S

In [58]:
P = np.array([[0.85, 0.15, 0.00],
              [0.005, 0.99, 0.005],
              [0.01, 0.04, 0.95]])
S = np.array([-5, 0, 5])

N = 1000
K = int(N/(1/(1 - P[0, 0]))) + N/100
ms = 2
num_of_states = len(S)

t1 = time.time()
ts = mkv_to_ts(P, S, n=N)
t2 = time.time()

est_P, est_S = MCTS(ts, num_of_states)
t3 = time.time()

print("Time Series Length: " + str(N))
print("Generation took: " + str(t2 - t1))
print("MCTS took: " + str(t3 - t2))
print("np.linalg.norm(P - est_P): " + str(np.linalg.norm(P - est_P)))
print("np.linalg.norm(S - est_S): " + str(np.linalg.norm(S - est_S)))
print(est_S)
print(est_P)
print()

Time Series Length: 1000
Generation took: 0.046369075775146484
MCTS took: 0.027651071548461914
np.linalg.norm(P - est_P): 1.0470996556674712
np.linalg.norm(S - est_S): 7.718549574143191
[-0.720042    5.01085083  0.98137574]
[[0.47091413 0.01090899 0.51817689]
 [0.16326531 0.42857143 0.40816327]
 [0.30582524 0.01618123 0.67799353]]

